In [ ]:
import numpy as np
import uproot as uproot
import uproot3 as uproot3
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from tqdm.notebook import tqdm

import pickle

# NuWro From Fake Data

In [ ]:
nuwro_fake_data_true_MA = 1.03

In [ ]:
bdt_vars = [
    "nue_score",
    "numu_score",
    "numu_cc_flag"
]

eval_vars = [
    "run",
    "subrun",
    "event",
    "match_isFC",

    # these aren't present in the fake data file
    #"truth_nuEnergy",
    #"truth_nuPdg",
    #"truth_isCC",
    #"truth_vtxInside",
    #"match_completeness_energy",
    #"truth_energyInside",
    #"weight_cv",
    #"weight_spline",
]

eval_data_vars = [
    "match_isFC",
]

kine_vars = [
    "kine_reco_Enu",
]

pf_vars = [
    "reco_muonMomentum",
    #"truth_muonMomentum",
]

pf_data_vars = [
    "reco_muonMomentum",
]

loc = "/Users/leehagaman/data/processed_checkout_rootfiles/"

f = uproot3.open(loc + "checkout_fakedata_nuwro_run1.root")["wcpselection"] # run 1 is numuCC only
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run1_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run1_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run1_df["file"] = "nuwro_fake_run1"

f = uproot3.open(loc + "checkout_fakedata_nuwro_run2.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run2_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run2_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run2_df["file"] = "nuwro_fake_run2"

f = uproot3.open(loc + "checkout_fakedata_nuwro_run3.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nuwro_fake_run3_vars_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nuwro_fake_run3_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nuwro_fake_run3_df["file"] = "nuwro_fake_run3"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval

wc_nuwro_fake_df = pd.concat([nuwro_fake_run1_df, nuwro_fake_run2_df, nuwro_fake_run3_df], axis=0, sort=False)

wc_nuwro_fake_df


In [ ]:
f = uproot.open(f"/Users/leehagaman/data/pelee_files/high_stat_prodgenie_bnb_nu_overlay_DetVar_Run1_NuWro_reco2_reco2.root")["nuselection"]["NeutrinoSelectionFilter"]


for item in f.items():
    if True:
        print(item)
    elif "nu" in item[0]:
        print(item)
    elif "q" in item[0] or "Q" in item[0]:
        print(item)


In [ ]:
for r in [1,2,3]:

    f = uproot.open(f"/Users/leehagaman/data/pelee_files/high_stat_prodgenie_bnb_nu_overlay_DetVar_Run{r}_NuWro_reco2_reco2.root")["nuselection"]["NeutrinoSelectionFilter"]

    run = f["run"].array()
    subrun = f["sub"].array()
    event = f["evt"].array()
    nu_pdg = f["nu_pdg"].array()
    interaction = f["interaction"].array()
    true_nu_energy = f["nu_e"].array() * 1000.
    true_lep_energy = f["lep_e"].array() * 1000.
    true_lep_theta = f["theta"].array()

    if r == 1:
        print(f"creating df with {len(run)} entries")
        pelee_nuWro_truth_df = pd.DataFrame({"run": run, "subrun": subrun, "event": event, 
                                             "pl_true_nu_pdg": nu_pdg,
                                             "pl_true_nu_energy": true_nu_energy, "pl_true_lep_energy": true_lep_energy, "pl_true_lep_theta": true_lep_theta,
                                             "pl_interaction_type": interaction})
    else:
        print(f"appending df with {len(run)} entries")
        pelee_nuWro_truth_df = pd.concat([pelee_nuWro_truth_df, pd.DataFrame({"run": run, "subrun": subrun, "event": event, 
                                                                              "pl_true_nu_pdg": nu_pdg,
                                                                              "pl_true_nu_energy": true_nu_energy, "pl_true_lep_energy": true_lep_energy, "pl_true_lep_theta": true_lep_theta,
                                                                              "pl_interaction_type": interaction})])
        

# Q^2 = - (p_nu - p_mu)^2
#     = -p_nu^2 + 2 p_nu * p_mu - p_mu^2
#     = -m_nu^2 + 2 E_nu * E_mu - 2 * P_nu * P_mu * cos(theta) - m_mu^2
#     = 2 E_nu * E_mu - 2 * E_nu * P_mu * cos(theta) - m_mu^2
#     = 2 * E_nu * (E_mu - P_mu * cos(theta)) - m_mu^2

pelee_nuWro_truth_df["pl_true_lep_mass"] = [105.6583755 if abs(pdg) == 13 else 0.51099895069 if abs(pdg) == 11 else 0. for pdg in pelee_nuWro_truth_df["pl_true_nu_pdg"]]
pelee_nuWro_truth_df["pl_true_lep_KE"] = pelee_nuWro_truth_df["pl_true_lep_energy"] - pelee_nuWro_truth_df["pl_true_lep_mass"]
pelee_nuWro_truth_df["pl_true_lep_momentum"] = np.sqrt(pelee_nuWro_truth_df["pl_true_lep_energy"]**2 - pelee_nuWro_truth_df["pl_true_lep_mass"]**2)
pelee_nuWro_truth_df["pl_true_lep_costheta"] = np.cos(pelee_nuWro_truth_df["pl_true_lep_theta"])


pelee_nuWro_truth_df["pl_true_Q2"] = 2 * pelee_nuWro_truth_df["pl_true_nu_energy"] * (pelee_nuWro_truth_df["pl_true_lep_energy"] - pelee_nuWro_truth_df["pl_true_lep_momentum"] * pelee_nuWro_truth_df["pl_true_lep_costheta"]) - pelee_nuWro_truth_df["pl_true_lep_KE"]**2

pelee_nuWro_truth_df


In [ ]:
nuwro_fake_df = pd.merge(wc_nuwro_fake_df, pelee_nuWro_truth_df, on=["run", "subrun", "event"], how="inner")
nuwro_fake_df


# NuWro From XS Generator

In [ ]:
f = uproot.open("generator_files/samples/NuWroCard_CC_Ar_uBFlux_1000.flat.root")
f.items()